In [2]:
import pandas as pd
import numpy as np

In [3]:
# import data
econ = pd.read_csv("economics.csv", parse_dates = ['date'])
econ['year'] = econ.date.apply(lambda x: x.to_pydatetime().year)
econ.sort_values(by = ['date'], ascending = True, inplace = True)
econ.head(5)

,date,pce,pop,psavert,uempmed,unemploy,year
0,1967-06-30,507.8,198712,9.8,4.5,2944,1967
1,1967-07-31,510.9,198911,9.8,4.7,2945,1967
2,1967-08-31,516.7,199113,9.0,4.6,2958,1967
3,1967-09-30,513.3,199311,9.8,4.9,3143,1967
4,1967-10-31,518.5,199498,9.7,4.7,3066,1967


In [4]:
# min and max pce by year
aggregations = {'pce': ['min', 'max']}
econ2 = econ.groupby('year', as_index = False)
econ2 = econ2.agg(aggregations)
econ2.columns = ['_'.join([y for y in list(x) if y != '']) for x in econ2.columns.ravel()]
econ2.head(5)

,year,pce_min,pce_max
0,1967,507.8,532.0
1,1968,534.7,584.9
2,1969,590.2,630.4
3,1970,634.0,678.0
4,1971,681.3,733.7


In [5]:
# periods with highest pce drawdown
econ3 = econ.copy()
econ3['pce_max'] = econ3.pce.cummax()
econ3['pce_drawdown'] = (1 - econ3.pce / econ3.pce_max)*100
econ3.sort_values(by = ['pce_drawdown'], ascending = False, inplace = True)
econ3.head(5)

,date,pce,pop,psavert,uempmed,unemploy,year,pce_max,pce_drawdown
234,1986-12-31,2959.7,241784,8.8,6.9,7892,1986,3019.5,1.980460
232,1986-10-31,2948.5,241467,8.1,7.1,8159,1986,2989.8,1.381363
219,1985-09-30,2762.3,239113,8.9,7.1,8298,1985,2799.7,1.335857
231,1986-09-30,2951.6,241274,7.5,7.0,8243,1986,2989.8,1.277677
410,2001-08-31,7012.7,286017,4.2,7.2,7142,2001,7098.6,1.210098


In [6]:
# calculate pce growth for each year
econ4 = econ.copy()
econ4['pce_growth'] = econ4.pce / econ4.pce.shift(1) # growth vs. previous period
aggregations = {'pce_growth': [lambda x: x.cumprod().tolist()[-1]]}
econ4 = econ4.groupby('year', as_index = False)
econ4 = econ4.agg(aggregations)
econ4.columns = [x[0] for x in econ4.columns.ravel()]
econ4.head(5)

,year,pce_growth
0,1967,1.047657
1,1968,1.099436
2,1969,1.077791
3,1970,1.075508
4,1971,1.082153


In [7]:
# filter to last data point for each year
econ5 = econ.sort_values(by = ['date'], ascending = False)
econ5 = econ5.groupby('year').first()
econ5.head(5)

,date,pce,pop,psavert,uempmed,unemploy
year,,,,,,
1967,1967-12-31,532.0,199808,9.0,5.1,2878
1968,1968-12-31,584.9,201760,7.1,4.4,2718
1969,1969-12-31,630.4,203849,8.3,4.6,3201
1970,1970-12-31,678.0,206466,10.0,6.2,4986
1971,1971-12-31,733.7,208917,9.1,6.2,5019
